# T5 - Juan Luis Baldelomar Cabrera

In [17]:
import nltk 
import numpy as np
import nltk
from nltk.probability import FreqDist
from nltk import TweetTokenizer
from nltk.corpus import stopwords

In [18]:
def load_data(filename, labels_filename):
    file = open(filename, 'r')
    labels_file = open(labels_filename, 'r')
    tweets = file.read()
    labels = labels_file.read()
    documents = tweets.split('\n')
    labels = labels.split('\n')
    documents.pop(-1)
    labels.pop(-1)
    return documents, labels

In [19]:
documents, labels = load_data('data/mex_train.txt', 'data/mex_train_labels.txt')
val_documents, val_labels = load_data('data/mex_val.txt', 'data/mex_val_labels.txt')

In [42]:
def tokenize_documents(documents, tokenizer=None):
    # default tokenizer
    tokenizer = TweetTokenizer().tokenize if tokenizer == None else tokenizer 
    documents_tokenized = []
    # tokenize each document
    for doc in documents:
        documents_tokenized.append(tokenizer(doc.lower()))
    return documents_tokenized

def get_vocabulary(tokenized_docs, n):
    tokens = [token for doc in tokenized_docs for token in doc]
    unique_tokens = FreqDist(tokens).most_common(n)
    return [token for token, _ in unique_tokens]

def word2ids(vocabulary):
    word2id = {}
    id2word = {}
    
    # build both dictionaries
    for i, word in enumerate(vocabulary):
        word2id[word] = i
        id2word[i] = word
    
    # add special tokens
    n = len(word2id)
    word2id['<s>']   = n 
    word2id['</s>']  = n + 1
    word2id['<unk>'] = n + 2
    id2word[n]       = '<s>'
    id2word[n + 1]   = '</s>'
    id2word[n + 2]   = '<unk>'
    
    return word2id, id2word

In [46]:
class NGramBuilder:
    def __init__(self, tokenizer=None, embeddings=None):
        self.tokenizer = tokenizer
        self.embeddings = embeddings
        self.word2id = None
        self.id2word = None
            
    def fit(self, documents, N, t=10000):
        self.N = N
        # tokenize, get vocabulary and word2id and ids2word dicts
        tokenized_docs = tokenize_documents(documents, self.tokenizer)
        vocabulary = get_vocabulary(tokenized_docs, t)
        self.word2id, self.id2word = word2ids(vocabulary)
        return self._transform(tokenized_docs)
    
    def get_vocabulary(self):
        return set(self.word2id.keys())
    
    def transform(self, documents):
        # tokenize, get vocabulary and word2id and ids2word dicts
        tokenized_docs = tokenize_documents(documents, self.tokenizer)
        return self._transform(tokenized_docs)
    
    def inverse(self, docs_as_ids):
        return [list(map(self.id2word.get, doc)) for doc in docs_as_ids]
    
    def _transform(self, tokenized_docs):
        N = self.N
        # docs and labels lists
        ngram_docs, ngram_targs = [], []
        # traverse each doc
        for doc in tokenized_docs:
            # add padding
            doc = ['<s>']*(N - 1)  + doc + ['</s>']
            # empty ngram and targets
            ngram_doc, ngram_tar = [], []
            # build list of ids from word2id dict      # DEFAULT VALUE of dict.get
            ids = list(map(self.word2id.get, doc,  [self.word2id['<unk>']] * len(doc)))
            
            # traverse each word as center and build ngrams
            for i in range(N-1, len(doc)):    
                ngram_doc.append(ids[i-(N-1): i])
                ngram_tar.append(ids[i])    
            # append document and labels
            ngram_docs.append(ngram_doc)
            ngram_targs.append(ngram_tar)
        
        return ngram_docs, ngram_targs

In [47]:
ngram_builder = NGramBuilder()
ngram_docs, ngram_labels = ngram_builder.fit(documents, N=3)

In [54]:
documents[0]

'lo peor de todo es que no me dan por un tiempo y luego vuelven estoy hasta la verga de estl'

In [53]:
ngram_builder.inverse(ngram_labels)[0]

['lo',
 'peor',
 'de',
 'todo',
 'es',
 'que',
 'no',
 'me',
 'dan',
 'por',
 'un',
 'tiempo',
 'y',
 'luego',
 'vuelven',
 'estoy',
 'hasta',
 'la',
 'verga',
 'de',
 'estl',
 '</s>']